<a href="https://colab.research.google.com/github/atick-faisal/Hand-Gesture-Recognition/blob/dev/Spatial-Path-Analysis/Spatial_Path_Transfer_Learning_BW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import gc
import cv2
import time
import joblib
import shutil
import tarfile
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score, precision_score, confusion_matrix
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, GlobalAvgPool2D, Dropout, Flatten, concatenate
from tensorflow.keras.losses import CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers, regularizers
from tensorflow.keras.models import Model

from scipy.signal import butter, lfilter

from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.utils import to_categorical

tf.__version__

'2.7.0'

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# -------- TEST USER ----------- #

TEST_USER      = '010'
DATASET_ID     = '1p0CSRb9gax0sKqdyzOYVt-BXvZ4GtrBv'

# BASE_DIR       = '../Dataset/'

# Google Drive
BASE_DIR       = '.'
DATA_DIR       = 'Sensor-Data/'
BW_IMG_DIR     = 'BW-Spatial-Path-Images/'
RGB_IMG_DIR    = 'RGB-Spatial-Path-Images/'
CHANNELS_DIR   = 'Channels/'
IMG_SIZE       = (3, 3) # INCHES

IMG_DIR        = 'BW-Spatial-Path-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007', '008', '009',
                  '010', '011', '012', '013', '014', '015', '016', '017', '018',
                  '019', '020', '021', '022', '023', '024', '025']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

PLANES         = ['XY', 'YZ', 'ZX']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
DATASET_LEN    = 4000
TRAIN_LEN      = 3840
TEST_LEN       = 160

EPOCHS         = 50
LEARNING_RATE  = 0.001
DECAY          = 0.0

DT             = 0.01
SHAPES         = 100
CUT_OFF        = 3.0
ORDER          = 4
FS             = 100

WINDOW_LEN     = 150

CONFIG         = '_L_7_S_160x160_E_7'
CHANNELS_GROUP = 'DYNAMIC_ACC_ONLY_'

XY_WEIGHTS     = np.array([0.91, 0.75, 0.61, 0.63, 0.51, 0.66, 0.81, 0.65, 0.65, 0.31,
                           0.66, 0.29, 0.34, 0.64, 0.64, 0.31])
YZ_WEIGHTS     = np.array([0.73, 0.71, 0.70, 0.79, 0.76, 0.38, 0.80, 0.61, 0.58, 0.73,
                           0.49, 0.26, 0.26, 0.52, 0.59, 0.54])
ZX_WEIGHTS     = np.array([0.33, 0.66, 0.51, 0.54, 0.37, 0.51, 0.71, 0.30, 0.75, 0.41,
                           0.40, 0.27, 0.24, 0.61, 0.36, 0.49])

In [4]:
class LowPassFilter(object): 
    def butter_lowpass(cutoff, fs, order):
        nyq = 0.5 * fs
        normal_cutoff = cutoff / nyq
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        return b, a

    def apply(data, cutoff=CUT_OFF, fs=FS, order=ORDER):
        b, a = LowPassFilter.butter_lowpass(cutoff, fs, order=order)
        y = lfilter(b, a, data)
        return y

In [5]:
#--------------------- Download util for Google Drive ------------------- #

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
        
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
        
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk:
                f.write(chunk)

def download_data(fid, destination):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(destination)
        print('√')
    except:
        print('✕')
        
    print('creating data directory ... ', end='')
    os.mkdir(destination)
    print('√')
    
    print('downloading dataset from the repository ... ', end='')
    filename = os.path.join(destination, 'dataset.tar.xz')
    try:
        download_file_from_google_drive(fid, filename)
        print('√')
    except:
        print('✕')
        
    print('extracting the dataset ... ', end='')
    try:
        tar = tarfile.open(filename)
        tar.extractall(destination)
        tar.close()
        print('√')
    except:
        print('✕')   

In [6]:
# # ------- Comment This if already downloaded -------- #

# destination = os.path.join(BASE_DIR, DATA_DIR)
# download_data(DATASET_ID, destination)

In [7]:
def clean_dir(path):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(path)
        print('√')
    except:
        print('✕')
    
    print('creating ' + path + ' directory ... ', end='')
    os.mkdir(path)
    print('√')

def extract_channels():
    channels_dir = os.path.join(BASE_DIR, CHANNELS_DIR)
    clean_dir(channels_dir)
        
    for user in USERS:
    # for gesture in GESTURES:
        print('Processing data for user ' + user, end=' ')
        # print(f"processing data for gesture {gesture} ", end="..." )
        
        X = []
        y = []
        first_time = True
        
        for gesture in GESTURES:
        # for user in USERS:
              
            user_dir = os.path.join(BASE_DIR, DATA_DIR, user)
            gesture_dir = os.path.join(user_dir, gesture + '.csv')

            dataset = pd.read_csv(gesture_dir)

            dataset['flex_1'] = dataset['flex_1'].rolling(3).median()
            dataset['flex_2'] = dataset['flex_2'].rolling(3).median()
            dataset['flex_3'] = dataset['flex_3'].rolling(3).median()
            dataset['flex_4'] = dataset['flex_4'].rolling(3).median()
            dataset['flex_5'] = dataset['flex_5'].rolling(3).median()

            dataset.fillna(0, inplace=True)

            # flex = ['flex_1', 'flex_2', 'flex_3', 'flex_4', 'flex_5']
            # max_flex = dataset[flex].max(axis=1)
            # max_flex.replace(0, 1, inplace=True)
            # dataset[flex] = dataset[flex].divide(max_flex, axis=0)
            
            flx1 = dataset['flex_1'].to_numpy().reshape(-1, WINDOW_LEN)
            flx2 = dataset['flex_2'].to_numpy().reshape(-1, WINDOW_LEN)
            flx3 = dataset['flex_3'].to_numpy().reshape(-1, WINDOW_LEN)
            flx4 = dataset['flex_4'].to_numpy().reshape(-1, WINDOW_LEN)
            flx5 = dataset['flex_5'].to_numpy().reshape(-1, WINDOW_LEN)
            
            accx = dataset['ACCx'].to_numpy()
            accy = dataset['ACCy'].to_numpy()
            accz = dataset['ACCz'].to_numpy()
            
            accx = LowPassFilter.apply(accx).reshape(-1, WINDOW_LEN)
            accy = LowPassFilter.apply(accy).reshape(-1, WINDOW_LEN)
            accz = LowPassFilter.apply(accz).reshape(-1, WINDOW_LEN)
            
            gyrx = dataset['GYRx'].to_numpy()
            gyry = dataset['GYRy'].to_numpy()
            gyrz = dataset['GYRz'].to_numpy()
            
            gyrx = LowPassFilter.apply(gyrx).reshape(-1, WINDOW_LEN)
            gyry = LowPassFilter.apply(gyry).reshape(-1, WINDOW_LEN)
            gyrz = LowPassFilter.apply(gyrz).reshape(-1, WINDOW_LEN)
            
            accm = np.sqrt(accx ** 2 + accy ** 2 + accz ** 2)
            gyrm = np.sqrt(gyrx ** 2 + gyry ** 2 + gyrz ** 2)
            
            g_idx = GESTURES.index(gesture)
            labels = np.ones((accx.shape[0], 1)) * g_idx
            
            channels = np.stack([
                flx1, flx2, flx3, flx4, flx5,
                accx, accy, accz
            ], axis=-1)
            
            if first_time == True:
                X = channels
                y = labels
                first_time = False
            else:
                X = np.append(X, channels, axis=0)
                y = np.append(y, labels, axis=0)
            
        
        x_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_X.joblib')
        y_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_y.joblib')
        joblib.dump(X, x_path)
        joblib.dump(y, y_path)
        
        print('√')
        

In [8]:
# ------------- Spatial Path Image Generation ----------- #

def clean_dir(path):
    print('cleaning already existing files ... ', end='')
    try:
        shutil.rmtree(path)
        print('√')
    except:
        print('✕')
    
    print('creating ' + path + ' directory ... ', end='')
    os.mkdir(path)
    print('√')
    
# ----------- Spatial Path Vector Calculation ----------- #

def get_displacement(acc):
    v = np.zeros(acc.shape)
    d = np.zeros(acc.shape)
    for i in range(acc.shape[0] - 1):
        v[i + 1] = v[i] + acc[i] * DT
        d[i + 1] = v[i] * DT + 0.5 * acc[i] * DT * DT
        
    return d

def write_image(x, y, path):
    fig, ax = plt.subplots(frameon=True, figsize=(3, 3))
    ax.axis('off')
    plt.scatter(x, y, s=SHAPES, c='black')
    plt.savefig(path)
    plt.close()

def generate_bw_images():
    image_dir = os.path.join(BASE_DIR, BW_IMG_DIR)
    clean_dir(image_dir)
    
    for plane in PLANES:
        print('processing spatial path images for ' + plane + ' plane ... ', end='')
        plane_dir = os.path.join(image_dir, plane)
        os.mkdir(plane_dir)
        
        # for gesture in GESTURES:
        #     os.mkdir(os.path.join(plane_dir, gesture))

        for user in USERS:
            os.mkdir(os.path.join(plane_dir, user))
    
            # for user in USERS:
            for gesture in GESTURES:
                os.mkdir(os.path.join(plane_dir, user, gesture))
                user_dir = os.path.join(BASE_DIR, DATA_DIR, user)
                gesture_dir = os.path.join(user_dir, gesture + '.csv')
                
                accx = pd.read_csv(gesture_dir)['ACCx'].to_numpy()
                accy = pd.read_csv(gesture_dir)['ACCy'].to_numpy()
                accz = pd.read_csv(gesture_dir)['ACCz'].to_numpy()

                x = get_displacement(accx).reshape(-1, 150)
                y = get_displacement(accy).reshape(-1, 150)
                z = get_displacement(accz).reshape(-1, 150)

                count = 0
                for i in range(x.shape[0]):
                    # image_name = 'u' + user + '_g' + '{:0>2d}'.format(GESTURES.index(gesture)) + \
                    #              '_s' + '{:0>7d}'.format(count) + '_p' + plane + '.jpg'
                    image_name = "{:0>7d}".format(count) + ".jpg"
                    path = os.path.join(plane_dir, user, gesture, image_name)
                    
                    if plane == 'XY':
                        write_image(x[i, :], y[i, :], path)
                    elif plane == 'YZ':
                        write_image(y[i, :], z[i, :], path)
                    else:
                        write_image(z[i, :], x[i, :], path)

                    count = count + 1
            
        print('√')

In [9]:
# extract_channels()
# generate_bw_images()

In [10]:
def load_data(plane, test_user):
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3), dtype='uint8')
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3), dtype='uint8')
    y_train = np.zeros((TRAIN_LEN, 1), dtype='uint8')
    y_test = np.zeros((TEST_LEN, 1), dtype='uint8')
    
    train_count = 0
    test_count = 0
        
    # for gesture in GESTURES:
    for user in USERS:
        print('loading data for user ' + user + ' on the ' + plane + ' plane ... ', end='')
        user_dir = os.path.join(BASE_DIR, IMG_DIR, plane, user)

        for gesture in GESTURES:
            gesture_dir = os.path.join(user_dir, gesture)

            # for filename in os.listdir(path):

            for count in range(10):
                image_name = "{:0>7d}".format(count) + ".jpg"
                img = cv2.imread(os.path.join(gesture_dir, image_name))
                resized = cv2.resize(img, IMG_SIZE)
                # resized = np.expand_dims(resized, axis=-1)
                # label = int(filename[6:8])
                if user != test_user:
                    X_train[train_count, :] = resized
                    y_train[train_count, 0] = GESTURES.index(gesture)
                    train_count = train_count + 1
                else:
                    X_test[test_count, :] = resized
                    y_test[test_count, 0] = GESTURES.index(gesture)
                    test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

def load_and_save_data(plane):
    X = np.zeros((DATASET_LEN, IMG_LEN, IMG_LEN, 1))
    y = np.zeros((DATASET_LEN, 1))
    
    train_count = 0
    test_count  = TRAIN_LEN
        
    for gesture in GESTURES:
        print('loading data for ' + gesture + ' gesture on the ' + plane + ' plane ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, plane, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename), cv2.IMREAD_GRAYSCALE)
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X[train_count, :] = resized
                y[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X[test_count, :] = resized
                y[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')

    joblib.dump(X, BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    joblib.dump(y, BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')

def load_data_from_joblib(plane):
    print('Loading data for ' + plane + ' plane ... ', end='')
    X = joblib.load(BASE_DIR + 'X_BW_' + plane + str(IMG_SIZE) + '.joblib')
    y = joblib.load(BASE_DIR + 'Y_BW_' + plane + str(IMG_SIZE) + '.joblib')
    test_user = int(TEST_USER)
    X_train = X[:TRAIN_LEN, :, :, :]
    y_train = y[:TRAIN_LEN, :]
    X_test = X[TRAIN_LEN:, :, :, :]
    y_test = y[TRAIN_LEN:, :]

    print('√')

    return X_train, X_test, y_train, y_test

In [11]:
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX')

# Save to Google  Drive
# load_and_save_data('XY')X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
# load_and_save_data('YZ')
# load_and_save_data('ZX')

# Load from Google Drive
# X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data_from_joblib('XY')
# X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data_from_joblib('YZ')
# X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data_from_joblib('ZX')

In [12]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [13]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [14]:
global_average_layer = GlobalAvgPool2D()
prediction_layer = Dense(len(GESTURES))

In [15]:
from tensorflow.python.keras.layers.pooling import MaxPooling2D

def get_conv_block_1D():
    input = Input(shape=(150, 1))
    x = BatchNormalization()(input)
    x = Conv1D(filters=8, kernel_size=3, activation='relu', padding='valid')(x)
    x = Conv1D(filters=16, kernel_size=3, activation='relu', padding='valid')(x)
    x = MaxPooling1D(2)(x)
    x = Conv1D(filters=16, kernel_size=3, activation='relu', padding='valid')(x)
    x = Conv1D(filters=16, kernel_size=3, activation='relu', padding='valid')(x)
    x = MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dense(50, activation='relu')(x)

    return input, x

def get_conv_block_2D():
    input = tf.keras.layers.Input(shape=IMG_SHAPE)
    # x = data_augmentation(input)
    x = preprocess_input(input)
    x = base_model(x, training=False)
    x = global_average_layer(x)

    # x = layers.Conv2D(16, 3, padding="valid", kernel_regularizer=regularizers.l2(0.001),)(
    #     input
    # )
    # x = layers.BatchNormalization()(x)
    # x = tf.keras.activations.relu(x)
    # x = layers.MaxPooling2D()(x)
    # x = layers.Conv2D(32, 3, padding="valid", kernel_regularizer=regularizers.l2(0.001),)(
    #     x
    # )
    # x = layers.BatchNormalization()(x)
    # x = tf.keras.activations.relu(x)
    # x = layers.MaxPooling2D()(x)
    # x = layers.Conv2D(
    #     64, 3, padding="valid", kernel_regularizer=regularizers.l2(0.001),
    # )(x)
    # x = layers.BatchNormalization()(x)
    # x = tf.keras.activations.relu(x)
    # x = layers.Flatten()(x)

    return input, x


def get_stacked_model():
    inputs = []
    CNNs = []

    for i in range(5):
        input_i, CNN_i = get_conv_block_1D()
        inputs.append(input_i)
        CNNs.append(CNN_i)

    for i in range(3):
        input_i, CNN_i = get_conv_block_2D()
        inputs.append(input_i)
        CNNs.append(CNN_i)

    x = concatenate(CNNs, axis=-1)
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.2)(x)
    output = Dense(16, activation="softmax")(x)
    model = Model(inputs, output)
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    loss = SparseCategoricalCrossentropy(from_logits=False)
    model.compile(loss=loss, optimizer=opt, metrics=["accuracy"])
    
    return model


In [16]:
model = get_stacked_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 1)]     0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 150, 1)]     0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 150, 1)]     0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None, 150, 1)]     0           []                               
                                                                                              

In [17]:
# def get_model():
#     inputs = tf.keras.Input(shape=IMG_SHAPE)
#     #     x = data_augmentation(inputs)
#     x = preprocess_input(inputs)
#     x = base_model(x, training=False)
#     x = global_average_layer(x)
#     x = tf.keras.layers.Dropout(0.2)(x)
#     outputs = prediction_layer(x)
#     model = tf.keras.Model(inputs, outputs)
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE, decay=DECAY),
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         metrics=["accuracy"],
#     )
#     return model


In [ ]:

ACC = []
CM = []
HISTORY = []
logs = ''

for test_user in USERS:
    print('Processing results for user ' + test_user, end='... \n')
    
    X_train = []
    X_test = []
    y_train = []
    y_test = []
    
    first_time_train = True
    first_time_test = True

    for user in USERS:
        x_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_X.joblib')
        y_path = os.path.join(BASE_DIR, CHANNELS_DIR, CHANNELS_GROUP + user + '_y.joblib')
        X = joblib.load(x_path)
        y = joblib.load(y_path)

        if user == test_user:
            if first_time_train == True:
                first_time_train = False
                X_test = X
                y_test = y
                
            else:
                X_test = np.append(X_test, X, axis=0)
                y_test = np.append(y_test, y, axis=0)
                
        else:
            if first_time_test == True:
                first_time_test = False
                X_train = X
                y_train = y
                
            else:
                X_train = np.append(X_train, X, axis=0)
                y_train = np.append(y_train, y, axis=0)


    # X_train, y_train = shuffle(X_train, y_train)

    X_train_xy, X_test_xy, y_train_xy, y_test_xy = load_data('XY', test_user)
    X_train_yz, X_test_yz, y_train_yz, y_test_yz = load_data('YZ', test_user)
    X_train_zx, X_test_zx, y_train_zx, y_test_zx = load_data('ZX', test_user)

    X_train_xy, X_train_yz, X_train_zx, X_train, y_train = shuffle(
        X_train_xy, X_train_yz, X_train_zx, X_train, y_train
    )

    X_train_combined = np.split(X_train, 8, axis=-1)[:5] + [X_train_xy, X_train_yz, X_train_zx]
    X_test_combined = np.split(X_test, 8, axis=-1)[:5] + [X_test_xy, X_test_yz, X_test_zx]

    del X_train_xy, X_test_xy, y_train_xy, y_test_xy
    del X_train_yz, X_test_yz, y_train_yz, y_test_yz
    del X_train_zx, X_test_zx, y_train_zx, y_test_zx
    del X_train, X_test

    gc.collect()

    print(len(X_train_combined))

    callbacks = [
                 tf.keras.callbacks.EarlyStopping(
                    monitor='val_accuracy', 
                    patience=30, 
                    mode='max', 
                    restore_best_weights=True
                ),
    ]

    model = get_stacked_model()
    history = model.fit(
        X_train_combined, 
        y_train, 
        validation_data=(X_test_combined, y_test),
        epochs=300, 
        batch_size=64,
        callbacks=[callbacks]
    )
    _, accuracy = model.evaluate(X_test_combined, y_test, batch_size=64)
    y_pred = model.predict(X_test_combined)
    cm = confusion_matrix(y_test.ravel(), np.argmax(y_pred, axis=-1))

    CM.append(cm)
    HISTORY.append(history)

    accuracy = accuracy * 100
    print(f'%.2f %%' %(accuracy))
    logs = logs + 'Accuracy for user ' + str(test_user) + '... ' + str(accuracy) + '\n'
    ACC.append(accuracy)
    
AVG_ACC = np.mean(ACC)
STD = np.std(ACC)
print('------------------------------------')
print(f'Average accuracy %.2f +/- %.2f' %(AVG_ACC, STD))

Processing results for user 001... 
loading data for user 001 on the XY plane ... √
loading data for user 002 on the XY plane ... √
loading data for user 003 on the XY plane ... √
loading data for user 004 on the XY plane ... √
loading data for user 005 on the XY plane ... √
loading data for user 006 on the XY plane ... √
loading data for user 007 on the XY plane ... √
loading data for user 008 on the XY plane ... √
loading data for user 009 on the XY plane ... √
loading data for user 010 on the XY plane ... √
loading data for user 011 on the XY plane ... √
loading data for user 012 on the XY plane ... √
loading data for user 013 on the XY plane ... √
loading data for user 014 on the XY plane ... √
loading data for user 015 on the XY plane ... √
loading data for user 016 on the XY plane ... √
loading data for user 017 on the XY plane ... √
loading data for user 018 on the XY plane ... √
loading data for user 019 on the XY plane ... √
loading data for user 020 on the XY plane ... √
load

In [ ]:
while True:
    pass

In [ ]:
# model = get_stacked_model()
# X_train_xy, X_train_yz, X_train_zx, y_train_xy = shuffle(
#     X_train_xy, X_train_yz, X_train_zx, y_train_xy
# )

# history = model.fit(
#     [X_train_xy, X_train_yz, X_train_zx],
#     y_train_xy,
#     validation_data=([X_test_xy, X_test_yz, X_test_zx], y_test_xy),
#     batch_size=32,
#     epochs=10,
# )


In [ ]:
# model_xy = get_model()
# X_train_xy, y_train_xy = shuffle(X_train_xy, y_train_xy)
# history_xy = model_xy.fit(X_train_xy, y_train_xy, validation_data=(X_test_xy, y_test_xy), epochs=EPOCHS)

In [ ]:
# # prob_xy = tf.keras.Sequential([model_xy, tf.keras.layers.Softmax()])
# # y_pred_xy = prob_xy.predict(X_test_xy)
# y_pred_xy = model_xy.predict(X_test_xy)
# y_pred = np.argmax(y_pred_xy, axis=1)
# print(classification_report(y_test_xy.ravel(), y_pred, zero_division=0))
# prc_xy = precision_score(y_test_xy.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# model_yz = get_model()
# X_train_yz, y_train_yz = shuffle(X_train_yz, y_train_yz)
# history_yz = model_yz.fit(X_train_yz, y_train_yz, validation_data=(X_test_yz, y_test_yz), epochs=EPOCHS)

In [ ]:
# # prob_yz = tf.keras.Sequential([model_yz, tf.keras.layers.Softmax()])
# # y_pred_yz = prob_yz.predict(X_test_yz)
# y_pred_yz = model_yz.predict(X_test_yz)
# y_pred = np.argmax(y_pred_yz, axis=1)
# print(classification_report(y_test_yz.ravel(), y_pred, zero_division=0))
# prc_yz = precision_score(y_test_yz.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# model_zx = get_model()
# X_train_zx, y_train_zx = shuffle(X_train_zx, y_train_zx)
# history_zx = model_zx.fit(X_train_zx, y_train_zx, validation_data=(X_test_zx, y_test_zx), epochs=EPOCHS)

In [ ]:
# # prob_zx = tf.keras.Sequential([model_zx, tf.keras.layers.Softmax()])
# # y_pred_zx = prob_zx.predict(X_test_zx)
# y_pred_zx = model_zx.predict(X_test_zx)
# y_pred = np.argmax(y_pred_zx, axis=1)
# print(classification_report(y_test_zx.ravel(), y_pred, zero_division=0))
# prc_zx = precision_score(y_test_zx.ravel(), y_pred, zero_division=0, average=None)
# tf.keras.backend.clear_session()

In [ ]:
# y_total = y_pred_xy + y_pred_yz + y_pred_zx
# y_pred = np.argmax(y_total, axis=1)
# report = classification_report(y_test_xy.ravel(), y_pred, zero_division=0)
# print(report)

In [ ]:
# config = '\n\nTEST_USER ' + TEST_USER + ' T: ' + str(int(time.time())) + '\n'
# underline = '=====================================\n'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'logs_sptl_bw' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(underline)
# f.write(report)
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_xy' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_xy, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_yz' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_yz, precision=2, max_line_width=100) + '\n')
# f.close()

In [ ]:
# config = TEST_USER + ' :'
# log_dir = os.path.join(BASE_DIR, LOG_DIR)
# if not os.path.exists(log_dir):
#     os.mkdir(log_dir)
# f = open(os.path.join(log_dir, 'prc_sptl_bw_zx' + CONFIG + '.txt'), 'a')
# f.write(config)
# f.write(np.array2string(prc_zx, precision=2, max_line_width=100) + '\n')
# f.close()